In [13]:
import pandas as pd

df = pd.read_excel('/Users/lee/Desktop/SPC.xlsx')


df['Seconds'] = [int(a) * 60 + int(b)  for a,b in df['Duration'].str.split(':')]

df['Date'] = [a  for a,b in df['Detected at'].str.split(' ')]

df['Seconds'] = df['Seconds'].astype(int)

# df.Date.astype('datetime64').dt.days

In [14]:
from openpyxl import load_workbook
import pandas as pd
from datetime import datetime
from openpyxl.styles import Font

df = pd.read_excel('/Users/lee/Desktop/SPC.xlsx')

df['Seconds'] = [int(a) * 60 + int(b)  for a,b in df['Duration'].str.split(':')]

df['Seconds'] = df['Seconds'].astype('int32')

df['Date'] = [a  for a,b in df['Detected at'].str.split(' ')]

d = {'nty': '南桃園',
     'best': '北視',
     'sh': '信和',
     'cy': '吉元',
     'cctv': '群健',
     'DataCenter' : 'DataCenter',
     'Total': 'Total',
     'Percentage': 'Percentage'}

month = datetime.now().strftime('%m')

dff = pd.DataFrame(columns=['<5 mins','5 mins ~ 1HR', '>1HR'], index=[v for k, v in d.items()])

for k, v in d.items():
    dff['<5 mins'][v] = df.loc[(df.Seconds < 300) & (df.Group == k) & (df.Action == 'Mitigation'), :].shape[0]
    
    dff['5 mins ~ 1HR'][v] = df.loc[(df.Seconds >= 300) & (df.Seconds < 3600) &
                                    (df.Group == k) & 
                                    (df.Action == 'Mitigation'), :].shape[0]
    
    dff['>1HR'][v] = df.loc[(df.Seconds > 3600) & (df.Group == k) & (df.Action == 'Mitigation'), :].shape[0]


dff.loc["Total"] = dff.iloc[0:6].sum().astype('int32')

dff.loc['Percentage'] = (dff.loc["Total"]/dff.loc["Total"].sum()).apply('{:.2%}'.format)

book = load_workbook('/Users/lee/Desktop/SPC.xlsx')
writer = pd.ExcelWriter('/Users/lee/Desktop/SPC.xlsx', engine='openpyxl')
writer.book = book
dff.to_excel(writer, sheet_name = 'Mitigation_Duration_2020_'+month)

fontStyle = Font(size = 16)
ws=book['Mitigation_Duration_2020_'+month]
ft = fontStyle

cell_dict = {"A":[1,2,3,4,5,6,7,8],
             "B":[1,2,3,4,5,6,7,8],
             "C":[1,2,3,4,5,6,7,8],
             "D":[1,2,3,4,5,6,7,8]}

for k,v in cell_dict.items():
    for i in range(1, 9):
        ws[k+str(i)].font = fontStyle
writer.save()
#==================================================================
                        
dff = pd.DataFrame(columns=['<5 mins','5 mins ~ 1HR', '>1HR'], index=[v for k, v in d.items()])

for k, v in d.items():
    dff['<5 mins'][v] = df.loc[(df.Seconds < 300) & (df.Group == k) & (df.Action == 'BlockHole'), :].shape[0]
    
    dff['5 mins ~ 1HR'][v] = df.loc[(df.Seconds >= 300) & (df.Seconds < 3600) &
                                    (df.Group == k) & 
                                    (df.Action == 'BlockHole'), :].shape[0]
    
    dff['>1HR'][v] = df.loc[(df.Seconds > 3600) & (df.Group == k) & (df.Action == 'BlockHole'), :].shape[0]


dff.loc["Total"] = dff.iloc[0:6].sum().astype('int32')

dff.loc['Percentage'] = (dff.loc["Total"]/dff.loc["Total"].sum()).apply('{:.2%}'.format)

book = load_workbook('/Users/lee/Desktop/SPC.xlsx')

writer = pd.ExcelWriter('/Users/lee/Desktop/SPC.xlsx', engine='openpyxl')
writer.book = book
dff.to_excel(writer, sheet_name = 'BlockHole_Duration_2020_'+month)

fontStyle = Font(size = 16)
ws=book['BlockHole_Duration_2020_'+month]
ft = fontStyle

cell_dict = {"A":[1,2,3,4,5,6,7,8],
             "B":[1,2,3,4,5,6,7,8],
             "C":[1,2,3,4,5,6,7,8],
             "D":[1,2,3,4,5,6,7,8]}

for k,v in cell_dict.items():
    for i in range(1, 9):
        ws[k+str(i)].font = fontStyle

writer.save()
writer.close()


dff


,<5 mins,5 mins ~ 1HR,>1HR
南桃園,9,22,0
北視,6,0,0
信和,0,0,0
吉元,0,0,0
群健,9,25,0
DataCenter,0,0,0
Total,24,47,0
Percentage,33.80%,66.20%,0.00%


In [15]:
from openpyxl import load_workbook
import pandas as pd
from datetime import date, timedelta, datetime

# dfff = pd.read_excel('/Users/lee/Desktop/SPC.xlsx')

# dfff['Date'] = [a  for a,b in df['Detected at'].str.split(' ')]

attack_type = df['Type'].unique().tolist()

# print(attack_type)

df["Detected at"].astype('datetime64')

max_date = df['Detected at'].max().split(" ")[0]
min_date = df['Detected at'].min().split(" ")[0]
aa = date.fromisoformat(max_date)
bb = date.fromisoformat(min_date)
result = aa - bb
result.days


d1 = date(aa.year, bb.month, bb.day)
all_days = [(d1 + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(result.days + 1)]

# print()
# print(all_days)

dfff = pd.DataFrame(columns=[attack_type[i] for i in range(len(attack_type))], 
                            index=[all_days[i] for i in range(len(all_days))])


for att in range(len(attack_type)):
    for i in range(len(all_days)):
        dfff[attack_type[att]][all_days[i]]= df.loc[(df.Type == attack_type[att]) & 
                                                        (df.Date == all_days[i]) &
                                                        (df.Action == "Mitigation"), :].shape[0]
        

dfff['Total'] = dfff.iloc[:].sum(axis=1).astype('int32')
          
book = load_workbook('/Users/lee/Desktop/SPC.xlsx')

writer = pd.ExcelWriter('/Users/lee/Desktop/SPC.xlsx', engine='openpyxl')
writer.book = book

ndfff = dfff.T

ndfff.to_excel(writer, sheet_name = 'Mitigation_DDoS type_2020_'+month)
    
writer.save()
writer.close()

print("below table is for number of mitigation events.")
print()
ndfff

below table is for number of mitigation events.



,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17
out-udp,2,0,10,4,19,20,1
udp,0,0,0,0,0,0,0
out-syn,9,12,21,8,19,14,3
frag,0,0,1,0,0,0,0
Total,11,12,32,12,38,34,4


In [1]:
dfff.info()

NameError: name 'dfff' is not defined

In [16]:
from openpyxl import load_workbook
import pandas as pd
from datetime import date, timedelta, datetime

# dfff = pd.read_excel('/Users/lee/Desktop/SPC.xlsx')

# dfff['Date'] = [a  for a,b in df['Detected at'].str.split(' ')]

attack_type = df['Type'].unique().tolist()

# print(attack_type)

df["Detected at"].astype('datetime64')

max_date = df['Detected at'].max().split(" ")[0]
min_date = df['Detected at'].min().split(" ")[0]
aa = date.fromisoformat(max_date)
bb = date.fromisoformat(min_date)
result = aa - bb
result.days


d1 = date(aa.year, bb.month, bb.day)
all_days = [(d1 + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(result.days + 1)]

# print()
# print(all_days)

dfff = pd.DataFrame(columns=[attack_type[i] for i in range(len(attack_type))], 
                            index=[all_days[i] for i in range(len(all_days))])



for att in range(len(attack_type)):
    for i in range(len(all_days)):
        dfff[attack_type[att]][all_days[i]]= df.loc[(df.Type == attack_type[att]) & 
                                                        (df.Date == all_days[i]) &
                                                        (df.Action == "BlockHole"), :].shape[0]
        

dfff['Total'] = dfff.iloc[:].sum(axis=1).astype('int32')
          
book = load_workbook('/Users/lee/Desktop/SPC.xlsx')

writer = pd.ExcelWriter('/Users/lee/Desktop/SPC.xlsx', engine='openpyxl')
writer.book = book

ndfff = dfff.T
ndfff.to_excel(writer, sheet_name = 'BlockHole_DDoS type_2020_'+month)
    
writer.save()
writer.close()
print("below table is for number of blockhole events.")
print()
ndfff

below table is for number of blockhole events.



,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17
out-udp,3,0,14,5,21,21,2
udp,2,0,0,0,0,0,1
out-syn,0,0,0,0,0,0,0
frag,0,0,0,1,1,0,0
Total,5,0,14,6,22,21,3


In [17]:
from openpyxl import load_workbook
import pandas as pd


nd = {'nty': '南桃園',
     'best': '北視',
     'sh': '信和',
     'cy': '吉元',
     'cctv': '群健',
      'DataCenter' : 'DataCenter',
     'Total': 'Total'}

ndf = pd.DataFrame(columns=[v for k,v in nd.items()], 
                   index=[all_days[i] for i in range(len(all_days))])

# del d['Percentage']

for key, value in nd.items():
    for i in range(len(all_days)):
        for att in range(len(attack_type)):
            ndf[value][all_days[i]]= df.loc[(df.Action == 'Mitigation') & 
                                            (df.Group == key) &
                                            (df.Date == all_days[i]), :].shape[0]


ndf['Total'] = ndf.iloc[:].sum(axis=1).astype('int32')

book = load_workbook('/Users/lee/Desktop/SPC.xlsx')

writer = pd.ExcelWriter('/Users/lee/Desktop/SPC.xlsx', engine='openpyxl')
writer.book = book

nndf = ndf.T

nndf.to_excel(writer, sheet_name = 'Mitigation events_'+month)
    
writer.save()
writer.close()
print("below table is for number of mitigation events for each SO.")
print()

nndf

below table is for number of mitigation events for each SO.



,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17
南桃園,0,0,2,1,1,4,0
北視,0,0,1,2,12,9,1
信和,0,0,5,2,8,3,0
吉元,0,0,3,0,0,0,0
群健,11,12,21,7,17,18,3
DataCenter,0,0,0,0,0,0,0
Total,11,12,32,12,38,34,4


In [18]:
from openpyxl import load_workbook
import pandas as pd


nd = {'nty': '南桃園',
     'best': '北視',
     'sh': '信和',
     'cy': '吉元',
     'cctv': '群健',
      'DataCenter' : 'DataCenter',
     'Total': 'Total'}

ndf = pd.DataFrame(columns=[v for k,v in nd.items()], 
                   index=[all_days[i] for i in range(len(all_days))])

# del d['Percentage']


for key, value in nd.items():
    for i in range(len(all_days)):
        for att in range(len(attack_type)):
            ndf[value][all_days[i]]= df.loc[(df.Action == 'BlockHole') & 
                                            (df.Group == key) &
                                            (df.Date == all_days[i]), :].shape[0]


ndf['Total'] = ndf.iloc[:].sum(axis=1).astype('int32')

book = load_workbook('/Users/lee/Desktop/SPC.xlsx')

writer = pd.ExcelWriter('/Users/lee/Desktop/SPC.xlsx', engine='openpyxl')
writer.book = book

nndf = ndf.T

nndf.to_excel(writer, sheet_name = 'BlockHole events_'+month)
    
writer.save()
writer.close()
print("below table is for number of blockhole events for each SO.")
print()

nndf

below table is for number of blockhole events for each SO.



,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17
南桃園,2,0,5,3,10,10,1
北視,1,0,4,0,1,0,0
信和,0,0,0,0,0,0,0
吉元,0,0,0,0,0,0,0
群健,2,0,5,3,11,11,2
DataCenter,0,0,0,0,0,0,0
Total,5,0,14,6,22,21,3
